In [5]:
# !git clone https://github.com/mattgroh/fitzpatrick17k

In [3]:
import pandas as pd
df = pd.read_csv('csv/fitzpatrick.csv')
missing = open('csv/missing.txt', 'r').read().split('\n')
df['exclude'] = df.apply(lambda r: True if r.md5hash in missing else False, axis=1)
df = df[~df.exclude]
print('pre', df.shape)

df['aggregated_fitzpatrick_scale'] = df.apply(lambda r: r.fitzpatrick_centaur if r.fitzpatrick_scale == -1 and r.fitzpatrick_centaur != -1 else r.fitzpatrick_scale, axis=1)
df['aggregated_fitzpatrick_scale'] = df.apply(lambda r: (r.aggregated_fitzpatrick_scale + r.fitzpatrick_centaur) / 2 if r.aggregated_fitzpatrick_scale != -1 and r.fitzpatrick_centaur != -1 else r.aggregated_fitzpatrick_scale, axis=1)
# df['aggregated_fitzpatrick_scale'] = df.aggregated_fitzpatrick_scale.round()

labels = sorted(df.label.unique())
mapping = dict(zip(labels, range(len(labels))))
df['target'] = df.label.map(mapping)

num_test_per_scale = 100
num_val_per_scale = 100
scales = sorted([x for x in df.aggregated_fitzpatrick_scale.unique() if x != -1])
test_df = pd.concat([df[df['aggregated_fitzpatrick_scale'] == scale].sample(n=num_test_per_scale, random_state=0) for scale in scales])
_df = df[~df.md5hash.isin(test_df.md5hash)]
val_df = pd.concat([_df[_df['aggregated_fitzpatrick_scale'] == scale].sample(n=num_val_per_scale, random_state=0) for scale in scales])
train_df = df[~df.md5hash.isin(pd.concat([val_df, test_df]).md5hash)]
print(*map(len, (train_df, val_df, test_df)))

train_df['split'] = 'train'
val_df['split'] = 'val'
test_df['split'] = 'test'
df = pd.concat([train_df, val_df, test_df])
df.loc[df.aggregated_fitzpatrick_scale == -1, 'split'] = 'exclude'
print('post', df.shape)
print(f'{len(train_df)=}')
print(f'{len(val_df)=}')
print(f'{len(test_df)=}')
print(train_df.aggregated_fitzpatrick_scale.value_counts())
print(df.split.value_counts())
df.to_csv('csv/fitzpatrick_v5.csv')

pre (16527, 14)
14327 1100 1100
post (16527, 14)
len(train_df)=14327
len(val_df)=1100
len(test_df)=1100
 1.5    2481
 1.0    2470
 2.0    2251
 3.0    1713
 2.5    1273
 3.5    1224
 4.0    1088
 4.5     607
 5.0     598
-1.0     289
 6.0     167
 5.5     166
Name: aggregated_fitzpatrick_scale, dtype: int64
train      14038
val         1100
test        1100
exclude      289
Name: split, dtype: int64


/var/folders/39/jd0hlp_j55d4kr11j2_nj2d00000gn/T/ipykernel_21941/3064837645.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['split'] = 'train'


In [4]:
pwd

'/Users/clu/repos/federated-conformal'

In [6]:
len(df.aggregated_fitzpatrick_scale.unique())

12

In [2]:
path_to_csv = 'fitzpatrick17k/fitzpatrick17k.csv'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv(path_to_csv)

FileNotFoundError: [Errno 2] No such file or directory: 'fitzpatrick17k/fitzpatrick17k.csv'

In [ ]:
df

In [18]:
from pathlib import Path
import requests

In [55]:
fitz_dir = Path('../data/fitzpatrick17k/')
image_dir = fitz_dir / 'images'
image_dir.mkdir(exist_ok=True)
missing_file = (fitz_dir / 'missing.txt')
missing_file.touch()

In [103]:
for i, row in df.iterrows():
    filename = row[0]
    image_url = row[7]
    if not isinstance(image_url, str):
        print('bad url', filename, image_url)
        with open(missing_file, 'a') as f:
            f.write(filename + '\n')
        continue
    ext = Path(image_url).suffix
    if (image_dir / f'{filename}{ext}').exists():
        continue
    response = requests.get(image_url, headers={'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})
    if not response.ok:
        print(filename, end='\t')
        print(response)
    else:
        with open(image_dir / f'{filename}{ext}','wb') as f:
            f.write(response.content)

bad url fc63008d9d80b373d2d8f84dfd034446 nan
bad url 480fd80167d7865ce9aafe14aba4ec9d nan
bad url acbc68bd9e35591b8344ab8b5e51f458 nan
bad url 93997108c3ae3beacc108f8829dc2302 nan
bad url 9d994f1278a9788612fa2af179328c31 nan
bad url 6712381a925124e968bcac89481c90e0 nan
bad url bb23fa10c71a57d17a8c7eb4773db237 nan
bad url d653edb6aa702f095a9b00550738a821 nan
bad url 8cfe54501fe00ddc74096b7cd67aa1e1 nan
bad url 85e11dd1c6f60d02f16516fdfac9c7f7 nan
bad url 5b2233d744c8b8d46c8577b69ba82a8f nan
bad url 3b713377bb9025c1ab6a184d788c940b nan
bad url 28126ac3ed45c0f1f282baa0c04232cd nan
bad url f0e1f501ec5f67e5ce9486a59304b4ab nan
bad url 267e230db9a29ddc9fb4c3b7a394a055 nan
bad url 2ed97fd85d7c7f94523d60fa1e3c3dfc nan
bad url 47e741a7840c2aa33f7c460fdad2fce2 nan
bad url 2851ef5f2ee2e0035cf316eeaeac217b nan
bad url aab39ed80019fefdabd07e7fa1125bc9 nan
bad url bb4fecc4701823a02f9c06f58e0da001 nan
bad url 0f119ca91147595c6299c830803cfb47 nan
bad url 04379559d6f119c6028c6790c2b3af1c nan
bad url ff

In [104]:
!ls ../data/fitzpatrick17k/images/ | wc -l

16527


In [105]:
small = []
issue = []
from PIL import Image
i = 0
for image_path in image_dir.glob('[!.]*'):
    try:
        img = Image.open(image_path)
        w, h = img.size
        pixels = np.array(img).sum()
        if w < 128 or h < 128 or pixels < 100_000:
            print(image_path.name, end='\t')
            print(img.size, end='\t')
            print(pixels)
            small.append(image_path)
    except Exception as e:
        print(image_path, e)
        issue.append(image_path)

68ab661c67a4b66b621eb0845f0fcccf.jpg	(130, 93)	4598402
7314c0c02f1d521c6b2b4ba955d369e8.jpg	(130, 99)	4881585
23fc2126bc1f8799fccfb195d2af6141.jpg	(133, 100)	5195807
23c779026faa7226e892c9048f43bf36.jpg	(117, 130)	3511061
c705f348b1035b9ad3d3fc2ee92ee768.jpg	(99, 130)	6332482
f968e591e15f47b544e551bc3cc5b8d3.jpg	(130, 120)	4506408
a7ef35e99387ff1227baced72467dc1f.jpg	(130, 85)	5478529
9d85c442ec9293c04423c8938cb36cad.jpg	(102, 130)	5413405
48700506ed2b3d5b0e5b4cbc6a12ad94.jpg	(130, 121)	3955616
dfae71d76e84de3d240a296bd40ff341.jpg	(130, 86)	5459411
8c4ed20439baa7bf0053b3abd8c481ee.jpg	(130, 89)	2971562
7da091d784090919f87bb8d5a1170830.jpg	(130, 120)	4926600
1dd06f10a962779a01ccae641e1c7e42.jpg	(130, 74)	2718669
742c83b0b5b1a14e8cddbef28108ae17.jpg	(130, 89)	5026602
4157d3f95f44ccc755f79cda16722730.jpg	(130, 74)	5377141
6bb966643b6a6f29047114b667aca8ec.jpg	(130, 117)	6781216
d04150a06562fb2c21073a745b42f7dc.jpg	(127, 192)	10912662
6bdffc50c482e5f7ecf1b1b325f6a172.jpg	(130, 108)	5524151


In [106]:
issue

[]

In [94]:
for p in issue:
    p.unlink()